# Accessing satellite data from AWS with Python

In [1]:
!pip uninstall folium -y

!pip install sentinelhub
!pip install geopandas
!pip install shapely
!pip install sentinelsat


Found existing installation: folium 0.14.0
Uninstalling folium-0.14.0:
  Successfully uninstalled folium-0.14.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=a292a01f89b1b332b673ce250a16a5309444b984a2c38d130ee6199ee731d44e
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.1 MB/s eta 0:00:00


This example notebook shows how to obtain Sentinel-2 imagery and additional data from [AWS S3 storage buckets](https://aws.amazon.com/s3/). The data at AWS is the same as original S-2 data provided by ESA.

The ```sentinelhub``` package supports obtaining data by specifying products or by specifying tiles. It can download data either to the same file structure as it is at AWS or it can download data into original ```.SAFE``` file structure [introduced by ESA](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/data-formats).

Before testing any of the examples below please check [Configuration paragraph](http://sentinelhub-py.readthedocs.io/en/latest/configure.html#amazon-s3-capabilities) for details about configuring AWS credentials and information about charges.

In [4]:
!pip install sentinelhub

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, SHConfig
import geopandas as gpd
from shapely.geometry import box
import pandas as pd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from collections import namedtuple

pd.set_option('display.max_rows', 5000)

ImportError: ignored

In [ ]:
# Getting reference Bounding Box


In [ ]:
minx = -48.44006632
xmax = -48.33143401
ymin = -1.46577103
ymax = -1.22102593

Bounding_Box = box(minx, ymin, xmax, ymax)

print('minx``, ``miny``, ``maxx``, ``maxy`')
print(Bounding_Box)

In [ ]:

def readUserKeys():
  with open("credentials.txt") as f:
    username, password = f.read().replace("\n", "").split(":")
  return username, password

username, password = readUserKeys()

api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and SciHub query keywords
footprint = Bounding_Box.wkt
products = api.query(footprint,
                     date=(date(2008, 1, 1), date(2017, 12, 29)),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))

print('N° of images queried: ', len(products))


In [ ]:

# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

products_df.head()


In [ ]:
# Sorting by cloudcover and data acquisition date
products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'datatakesensingstart'], ascending=[True, True])
products_df_sorted.shape

In [ ]:
import os, glob

def check_wild_file_existance(directory, basename):

    filenames = glob.glob(os.path.join(directory, basename + '*'))

    if len(filenames):
        return True
    else:
        return False

In [ ]:
minx, miny, maxx, maxy = np.round(Bounding_Box.bounds, 0).astype(int)

directory_path = r'IMAGENS_SENTINEL\S2\downloads\BB_minx{0}_miny{1}_maxx{2}_maxy{3}'.format(minx, miny, maxx, maxy)

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

In [ ]:

for index in products_df_sorted.index:
    # download sorted and reduced products

    product_info = api.get_product_odata(index)

    # making sure that the data is not yet downloaded locally:

    if not check_wild_file_existance(directory_path, product_info['title']):

        if product_info['Online']:
            print('Download start of \n\t {0}'.format(product_info['title']), end='\n'*2)

            # api.download(index, directory_path=directory_path)

        else:
            print('{} is off-online. Skipping.'.format(product_info['title']), end='\n'*2)

    else:
        print('{0} is already in folder.'.format(product_info['title']), '\n\t Skipping download', end='\n'*2)
